# New Features in MI Scripting Toolkit v1.5

## Deleting records (DeleteOrWithdrawIfLatestRecordVersion)

#### Let's start by creating a new record (Note: added receiveTimeout to the MISession)

In [1]:
import GRANTA_MIScriptingToolkit as gdl
from time import strftime
import datetime

session = gdl.GRANTA_MISession('http://localhost/mi_servicelayer/',autoLogon=True, receiveTimeout=5000)

raw_record_name = 'STK_1_5_Import_Test'
rec_name = raw_record_name + '_' + strftime("%Y-%m-%d %H:%M:%S")
table_name = 'Tensile Test Data'
subset_name = table_name
db_key = 'MI_Training'

import_service = session.dataImportService
browse_service = session.browseService
export_service = session.dataExportService

test_table_ref = gdl.TableReference(name=table_name, DBKey=db_key)
partial_table_ref = gdl.PartialTableReference(tableName=table_name)
subset_ref = gdl.SubsetReference(DBKey=db_key, partialTableReference=partial_table_ref, name=subset_name)
table_root_ref = browse_service.GetRootNode(gdl.GetRootNode(table=test_table_ref)).rootNode.recordReference

import_record = gdl.ImportRecord(recordName=rec_name, importAttributeValues=None, isFolder=False,
                                 existingRecord=table_root_ref, subsetReferences=[subset_ref])

import_request = gdl.SetRecordAttributesRequest(importRecords=[import_record])
imported_records = import_service.SetRecordAttributes(import_request).recordsImported
imported_record_ref = imported_records[0].recordReference
print('Imported record "{}" to table "{}"'.format(imported_records[0].longName, table_name))

Imported record "STK_1_5_Import_Test_2022-05-10 23:20:12" to table "Tensile Test Data"


#### Now let's delete it!

In [2]:
delete_record = gdl.DeleteOrWithdrawRecord(recordReference=imported_record_ref)
records_to_delete = [delete_record]

delete_records_request = gdl.DeleteOrWithdrawIfLatestRecordVersionRequest(deleteOrWithdrawRecords=records_to_delete)
delete_records_response = import_service.DeleteOrWithdrawIfLatestRecordVersion(delete_records_request)
print('Record {} was deleted'.format(imported_records[0].longName))

Record STK_1_5_Import_Test_2022-05-10 23:20:12 was deleted


## ResolveReferences

#### Let's see if we can modify a record...

In [3]:
modfied_record_guid = '4269f1bc-df6b-4a6d-870c-ef931728f37b'
modified_record_ref = gdl.RecordReference(DBKey=db_key, recordGUID=modfied_record_guid)

resolve_refs_request = gdl.ResolveReferencesRequest(entities=[modified_record_ref])

can_modify_int = browse_service.ResolveReferences(resolve_refs_request).entityResolutions[0].canWrite
if can_modify_int == 0:
    print('Yes, we can edit this record.')
else:
    print("No, we don't have permission to edit this record.")

Yes, we can edit this record.


## lastModifier and last modifiedDate pseudo-attributes

#### If we can modify the record, let's make a small change to it...

In [4]:
if can_modify_int == 0:
    attribute_project_notes = 'Project Notes'
    attribute_val = 'Project was updated: {0}'.format(datetime.datetime.now())
    attribute_ref = gdl.AttributeReference(name=attribute_project_notes, DBKey=db_key, partialTableReference=partial_table_ref)
    import_attribute_val = gdl.ImportAttributeValue(longTextDataValue=gdl.LongTextDataType(attribute_val),
                                                    attributeReference=attribute_ref) 
    update_record = gdl.ImportRecord(recordName=None,
                                     importRecordMode='Update',
                                     importAttributeValues=[import_attribute_val],
                                     existingRecord=modified_record_ref)

    update_request = gdl.SetRecordAttributesRequest(importRecords=[update_record])
    updated_records = import_service.SetRecordAttributes(update_request).recordsImported
    updated_record_ref = updated_records[0].recordReference
    updated_record_name = updated_records[0].longName
    print('Updated record "{}"'.format(updated_record_name))
else:
    print("Nothing to do - we can't modify the record!")

Updated record "MTS-615723"


#### Now let's retrieve the record creator, createdDate, lastModifier and modifiedDate

In [5]:
pseudo_creator = gdl.AttributeReference(pseudoAttribute=gdl.AttributeReference.MIPseudoAttributeReference.creator,
                                        DBKey=db_key, partialTableReference=partial_table_ref)
pseudo_created_date = gdl.AttributeReference(pseudoAttribute=gdl.AttributeReference.MIPseudoAttributeReference.createdDate,
                                             DBKey=db_key, partialTableReference=partial_table_ref)
pseudo_last_modifier = gdl.AttributeReference(pseudoAttribute=gdl.AttributeReference.MIPseudoAttributeReference.lastModifier,
                                              DBKey=db_key, partialTableReference=partial_table_ref)
pseudo_modified_date = gdl.AttributeReference(pseudoAttribute=gdl.AttributeReference.MIPseudoAttributeReference.modifiedDate,
                                              DBKey=db_key, partialTableReference=partial_table_ref)
attribute_refs = [pseudo_creator, pseudo_created_date, pseudo_last_modifier, pseudo_modified_date]

export_request = gdl.GetRecordAttributesByRefRequest(recordReferences=[modified_record_ref],
                                                     attributeReferences=attribute_refs)

exported_data = export_service.GetRecordAttributesByRef(export_request).recordData
for record in exported_data:
    creator = [a.shortTextDataType.value for a in record.attributeValues if a.attributeName == '[creator]'][0]
    created_date = [a.dateDataType.value for a in record.attributeValues if a.attributeName == '[createdDate]'][0]
    modifier = [a.shortTextDataType.value for a in record.attributeValues if a.attributeName == '[lastModifier]'][0]
    modified_date = [a.dateDataType.value for a in record.attributeValues if a.attributeName == '[modifiedDate]'][0]
    print('"{}" imported the record on {}'.format(creator, created_date))
    print('The record was last modified on {} by "{}"'.format(modified_date, modifier))

"Ansys Granta 1" imported the record on 
The record was last modified on  by "DOMAIN\username"


## GetUnitConversions

#### For the above record, let's export the Young's Modulus in the Metric system

In [6]:
attribute_name = "Young's Modulus (11-axis)"
unit_context_metric = gdl.UnitConversionContext(unitSystem='Metric')

modulus_attribute_ref = gdl.AttributeReference(name=attribute_name, DBKey=db_key, partialTableReference=partial_table_ref)
export_request = gdl.GetRecordAttributesByRefRequest(recordReferences=[modified_record_ref],
                                                     attributeReferences=[modulus_attribute_ref])

exported_data = export_service.GetRecordAttributesByRef(export_request).recordData
modulus_value = [[a.pointDataType.points[0].value for a in r.attributeValues] for r in exported_data][0][0]
modulus_metric_symbol = [[a.pointDataType.unitSymbol for a in r.attributeValues] for r in exported_data][0][0]
print("Young's modulus = {} {}".format(modulus_value, modulus_metric_symbol))

Young's modulus = 174.0004272460938 GPa


#### Now let's convert its value with all available conversions in Granta MI

In [7]:
import pandas

unit_conversions_request = gdl.GetUnitConversionsRequest(DBKey=db_key, unitSymbols=[modulus_metric_symbol])

source_units = browse_service.GetUnitConversions(unit_conversions_request).sourceUnits
conversion_targets = [c.conversions for c in source_units]
factors_and_offsets = [[(c.factor, c.offset) for c in c_target] for c_target in conversion_targets][0]
symbols = [[c.targetSymbol for c in c_target] for c_target in conversion_targets][0]
results = [modulus_value * factor + offset for factor, offset in factors_and_offsets]
equations = ['{} * {} + {}'.format(modulus_value, factor, offset) for factor, offset in factors_and_offsets]

factors, offsets = zip(*factors_and_offsets)
df = pandas.DataFrame([factors, offsets, equations, results])
df_flipped = df.transpose()
df_flipped.columns = ['factor', 'offset', 'equation', 'converted result']
df_flipped.index = symbols
df_flipped.style

,factor,offset,equation,converted result
10^6 psi,0.145038,0.000000,174.0004272460938 * 0.14503772518547858 + 0.0,25.236626
ksi,145.037725,0.000000,174.0004272460938 * 145.0377251854786 + 0.0,25236.626149
psi,145037.725185,0.000000,174.0004272460938 * 145037.72518547857 + 0.0,25236626.149075
MGO,125663.706106,0.000000,174.0004272460938 * 125663.70610560982 + 0.0,21865538.551704
Pa,1000000000.000000,0.000000,174.0004272460938 * 1000000000.0 + 0.0,174000427246.093811
MPa,1000.000000,0.000000,174.0004272460938 * 1000.0 + 0.0,174000.427246
J/m^3,1000000000.000000,0.000000,174.0004272460938 * 1000000000.0 + 0.0,174000427246.093811
MJ/m^3,1000.000000,0.000000,174.0004272460938 * 1000.0 + 0.0,174000.427246
erg/cm^3,10000000000.000002,0.000000,174.0004272460938 * 10000000000.000002 + 0.0,1740004272460.938477
ft.lbf/in^3,12086.477099,0.000000,174.0004272460938 * 12086.477098789885 + 0.0,2103052.179090
